In [36]:
import tensorflow as tf
from tensorflow import keras #for neural nets
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [12]:
physical_devices = tf.config.list_physical_devices('GPU')
physical_devices
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

[]

In [17]:
x = tf.constant(3)
x = tf.constant([[2,2], [3,4]])

x = tf.eye(3,3)
x = tf.ones((3,4))
x = tf.random.normal((3,3), mean=1, stddev=0.2)
x = tf.random.uniform((3,3), minval=0, maxval=1)
x = tf.cast(x, dtype=tf.int16) #convert between different types
print(x)

tf.Tensor(
[[0 0 0]
 [0 0 0]
 [0 0 0]], shape=(3, 3), dtype=int16)


In [10]:
a = tf.constant([2,3,4])
b = tf.constant([8,7,6])

x = tf.add(a,b)
x = a - b
x= a * b
e = tf.tensordot(a,b, axes=1)
e = tf.reduce_sum(a*b, axis=0)

s = x*3 #actions with each element in the tensor or matrix
print(s)

tf.Tensor([48 63 72], shape=(3,), dtype=int32)


In [15]:
x = tf.random.normal((3,3), mean=1, stddev=0.2)
y = tf.random.normal((3,3), mean=1, stddev=0.2)

z = tf.matmul(x,y)
z = x @ y
print(z)

tf.Tensor(
[[3.0279515 3.267978  2.4835138]
 [2.7245784 2.933443  2.2279365]
 [2.7958477 3.0220232 2.303628 ]], shape=(3, 3), dtype=float32)


In [18]:
# Indexing
x = tf.constant([0,1,2,3,4,5])
print(x[1:]) #except first

print(x[1:3])

tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int32)
tf.Tensor([1 2], shape=(2,), dtype=int32)


In [29]:
x = tf.constant([[0,1,2,3,4,5],[3,3,3,4,5,6],[3,33,34,4,5,6]])
print(x[0, :2])
print(x[0:2,:2])

#RESHAPING

x = tf.range(9)
x = tf.reshape(x, (3,3))

print(x)

tf.Tensor([0 1], shape=(2,), dtype=int32)
tf.Tensor(
[[0 1]
 [3 3]], shape=(2, 2), dtype=int32)


In [37]:
# Neural network on MNIST (numbers from 0 to 9)

(x_train, y_train), (x_test, y_test) = mnist.load_data()


11490434/11490434 [==============================] - 1s 0us/step


In [44]:
print(x_train.shape)

(60000, 28, 28)


In [45]:
x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0

In [53]:
# Sequential API of keras (convenient, not very flexible) - one imput mapped to one output

model = keras.Sequential(
    [
        keras.Input(shape=(28*28)),
        layers.Dense(512, activation='relu'), #activation function
        layers.Dense(256, activation='relu'), 
        layers.Dense(10)
    ]
)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), # we dont have softmax activation , Sparse categorical - labels are just an integer of the correct label, for categorical we need one Hot encoding
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'],
)

In [54]:
print(model.summary()) #is possible only when dimensions of the dataset are specified keras.Input(shape=(28*28))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               401920    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535818 (2.04 MB)
Trainable params: 535818 (2.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [49]:
model.fit(x_train, y_train, batch_size=32, epochs= 2, verbose=True)

Epoch 1/2
1875/1875 [==============================] - 31s 16ms/step - loss: 0.1837 - accuracy: 0.9449
Epoch 2/2
1875/1875 [==============================] - 33s 18ms/step - loss: 0.0801 - accuracy: 0.9754


In [50]:
model.evaluate(x_test, y_test, batch_size=32, verbose=True)

313/313 [==============================] - 1s 4ms/step - loss: 0.0863 - accuracy: 0.9744


[0.08626342564821243, 0.974399983882904]

In [56]:
# if it is possible sequential API should be used to build the model
model2 = keras.Sequential()

model2.add(keras.Input(shape=(784)))
model2.add(layers.Dense(512, activation='relu'))
print(model2.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
Total params: 401920 (1.53 MB)
Trainable params: 401920 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [58]:
#functional API (more flexible, multiple inputs x multiple outputs)

inputs= keras.Input(shape=(784))
x = layers.Dense(512, activation='relu', name='First_layer')(inputs)
x = layers.Dense(512, activation='relu', name='Second_layer')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model3 = keras.Model(inputs=inputs, outputs=outputs)

model3.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False), # we dont have softmax activation , Sparse categorical - labels are just an integer of the correct label, for categorical we need one Hot encoding
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'],
)

In [ ]:
print(model3.summary())

In [59]:
model3.fit(x_train, y_train, batch_size=32, epochs= 2, verbose=True)

Epoch 1/2
1875/1875 [==============================] - 55s 29ms/step - loss: 0.1829 - accuracy: 0.9453
Epoch 2/2
1875/1875 [==============================] - 45s 24ms/step - loss: 0.0789 - accuracy: 0.9753


In [60]:
model3.evaluate(x_test, y_test, verbose=True)

313/313 [==============================] - 1s 4ms/step - loss: 0.0863 - accuracy: 0.9745


[0.08629511296749115, 0.9745000004768372]

In [ ]:
model4 = keras.Model(inputs=model3.inputs, outputs=model3.layers[-2].output)